# Quantized Bonito Tutorial
This is a tutorial to set up and run a quantized version of [Bonito](https://github.com/BatsResearch/bonito) on a Google Colab T4 instance. The quantized model was graciously created by GitHub/HuggingFace user `alexandreteles` and we thank them for their contributions! The versions they created are:
 - [alexandreteles/bonito-v1-awq](https://huggingface.co/alexandreteles/bonito-v1-awq) (can be used directly with vllm)
 - [alexandreteles/bonito-v1-gguf](https://huggingface.co/alexandreteles/bonito-v1-gguf) (for llama.cpp inference)

In this tutorial, we demonstrate the `awq` version with the `transformers` package, which works well in a Colab environment.


First we clone into the repo and install the dependencies. This will take several minutes.

In [ ]:
!git clone https://github.com/BatsResearch/bonito.git
!pip install -e bonito/

To use this quantized model, we need to install the [AutoAWQ](https://github.com/casper-hansen/AutoAWQ) package, which deals with AWQ ([Activation-aware Weight Quantization](https://arxiv.org/abs/2306.00978)) models, such as the one we'll be using. AWQ is a quantization technique that treats different weight parameters differently based on their importance. To get it to work with Colab, we have to install the kernel from a specialized wheel so the CUDA versions match.

In [ ]:
!pip install autoawq
!git clone https://github.com/Boltuzamaki/AutoAWQ_kernels.git
!pip install AutoAWQ_kernels/builds/autoawq_kernels-0.0.6+cu122-cp310-cp310-linux_x86_64.whl

This cell includes the code to work with the quantized Bonito model, utilizing the `transformers` package. It's similar to the `Bonito` code made for `vllm` in the repo.

In [ ]:
from typing import Optional
from datasets import Dataset
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


SHORTFORM_TO_FULL_TASK_TYPES = {
    "exqa": "extractive question answering",
    "mcqa": "multiple-choice question answering",
    "qg": "question generation",
    "qa": "question answering without choices",
    "ynqa": "yes-no question answering",
    "coref": "coreference resolution",
    "paraphrase": "paraphrase generation",
    "paraphrase_id": "paraphrase identification",
    "sent_comp": "sentence completion",
    "sentiment": "sentiment",
    "summarization": "summarization",
    "text_gen": "text generation",
    "topic_class": "topic classification",
    "wsd": "word sense disambiguation",
    "te": "textual entailment",
    "nli": "natural language inference",
}


class QuantizedBonito():
    def __init__(self, model_name_or_path):
        self.model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True).cuda()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

    def generate_tasks(
        self,
        text_dataset: Dataset,
        context_col: str,
        task_type: str,
        sampling_params: dict,
        **kwargs,
    ):
        """
        Generates tasks using the Bonito model.

        This method takes a text dataset, a context column name,
        a task type, and sampling parameters, and generates tasks
        using the Bonito model. It processes the input dataset,
        generates outputs, collects multiple generations into
        one dataset object, and filters out the examples that
        cannot be parsed.

        Args:
            text_dataset (Dataset): The dataset that provides the text
                for the tasks.
            context_col (str): The name of the column in the dataset
                that provides the context for the tasks.
            task_type (str): The type of the tasks. This can be a
                short form or a full form.
            sampling_params (dict): The parameters for
                sampling.
            **kwargs: Additional keyword arguments.

        Returns:
            Dataset: The synthetic dataset with the generated tasks.
        """
        processed_dataset = self._prepare_bonito_input(
            text_dataset, task_type, context_col, **kwargs
        )

        outputs = self._generate_text(processed_dataset["input"], sampling_params)

        # collect multiple generations into one dataset object
        examples = []
        for i, example in enumerate(text_dataset.to_list()):
            output = outputs[i]
            example["prediction"] = output.strip()
            examples.append(example)

        synthetic_dataset = Dataset.from_list(examples)

        # filter out the examples that cannot be parsed
        synthetic_dataset = self._postprocess_dataset(
            synthetic_dataset, context_col, **kwargs
        )

        return synthetic_dataset

    def _generate_text(self, dataset: Dataset, sampling_params: dict):
        generated_texts = []

        for prompt in dataset:
            input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
            input_ids = input_ids.cuda()

            output = self.model.generate(
                input_ids,
                do_sample=True,
                **sampling_params
            )

            generated_text = self.tokenizer.decode(output[0][len(input_ids[0]):], skip_special_tokens=True)
            generated_texts.append(generated_text)

        return generated_texts


    def _prepare_bonito_input(
        self, context_dataset: Dataset, task_type: str, context_col: str, **kwargs
    ) -> Dataset:
        """
        Prepares the input for the Bonito model.

        This method takes a context dataset, a task type, and a context
        column name, and prepares the dataset for the Bonito model.
        If the task type is not recognized, it raises a ValueError.

        Args:
            context_dataset (Dataset): The dataset that provides the
                context for the task.
            task_type (str): The type of the task. This can be a
                short form or a full form. If the task type is not
                recognized, a ValueError is raised.
            context_col (str): The name of the column in the dataset
                that provides the context for the task.
            **kwargs: Additional keyword arguments.

        Returns:
            Dataset: The prepared dataset for the Bonito model.
        """
        # get the task type name
        if task_type in SHORTFORM_TO_FULL_TASK_TYPES.values():
            full_task_type = task_type
        elif task_type in SHORTFORM_TO_FULL_TASK_TYPES:
            full_task_type = SHORTFORM_TO_FULL_TASK_TYPES[task_type]
        else:
            raise ValueError(f"Task type {task_type} not recognized")

        def process(example):
            input_text = "<|tasktype|>\n" + full_task_type.strip()
            input_text += (
                "\n<|context|>\n" + example[context_col].strip() + "\n<|task|>\n"
            )
            return {
                "input": input_text,
            }

        return context_dataset.map(
            process,
            remove_columns=context_dataset.column_names,
            num_proc=kwargs.get("num_proc", 1),
        )

    def _postprocess_dataset(
        self, synthetic_dataset: Dataset, context_col: str, **kwargs
    ) -> Dataset:
        """
        Post-processes the synthetic dataset.

        This method takes a synthetic dataset and a context column
        name, and post-processes the dataset. It filters out
        examples where the prediction does not contain exactly two
        parts separated by "<|pipe|>", and then maps each example to a
        new format where the context is inserted into the first part of
        the prediction and the second part of the prediction is used as
        the output.

        Args:
            synthetic_dataset (Dataset): The synthetic dataset to be
                post-processed.
            context_col (str): The name of the column in the dataset
                that provides the context for the tasks.
            **kwargs: Additional keyword arguments.

        Returns:
            Dataset: The post-processed synthetic dataset.
        """
        synthetic_dataset = synthetic_dataset.filter(
            lambda example: len(example["prediction"].split("<|pipe|>")) == 2
        )

        def process(example):
            pair = example["prediction"].split("<|pipe|>")
            context = example[context_col].strip()
            return {
                "input": pair[0].strip().replace("{{context}}", context),
                "output": pair[1].strip(),
            }

        column_names = synthetic_dataset.column_names
        processed_synthetic_dataset = synthetic_dataset.map(
            process,
            remove_columns=column_names,
            num_proc=kwargs.get("num_proc", 1),
        )

        return processed_synthetic_dataset


This is where we load in the model and unannotated dataset. With them, we can generate a synthetic dataset of instructions. This example generates synthetic instructions from a subset of size 10 of the unannotated dataset.

In [ ]:
from datasets import load_dataset

# Initialize the Bonito model
bonito = QuantizedBonito("alexandreteles/bonito-v1-awq")

# load dataset with unannotated text
unannotated_text = load_dataset(
    "BatsResearch/bonito-experiment",
    "unannotated_contract_nli"
)["train"].select(range(10))

# Generate synthetic instruction tuning dataset
sampling_params = {'max_length':256, 'top_p':0.95, 'temperature':0.5, 'num_return_sequences':1}
synthetic_dataset = bonito.generate_tasks(
    unannotated_text,
    context_col="input",
    task_type="nli",
    sampling_params=sampling_params
)

print(synthetic_dataset)

Now go try it out with your own datasets! You can vary the `task_type` for different types of generated instructions.